In [ ]:
from downsampling import extract_single_shell


bval = 1000
bval_range = 20
directions = [6, 12, 18, 24, 32, 64]

path = '/home/mariano/Downloads/DTI'
dwi_in = '100307_DWI_processed_b1000.mif.gz'
dwi_in = os.path.join(path, dwi_in)

for i, dir_i in enumerate(directions):
    sub = os.path.basename(dwi_in).split('_')[0]
    print(' '.join([' '] * 300), end='\r')
    print('Subject {:} [{:02d}/{:02d}]'.format(
        sub, i + 1, len(dwi_files)
    ), end='\r')
    
    image = os.path.basename(dwi_in)

    print(' '.join([' '] * 300), end='\r')
    print('Reading the MIF file {:} [{:02d}/{:02d}]'.format(
        sub, i + 1, len(dwi_files)
    ), end='\r')
    mif = load_mrtrix(dwi_in)
    old_grad = mif.grad
    bvals = old_grad[:, -1]
    bvecs = old_grad[:, :-1]
    
    print(' '.join([' '] * 300), end='\r')
    print('Downsampling {:} [{:02d}/{:02d}]'.format(
        sub, i + 1, len(dwi_files)
    ), end='\r')
    lr_bvecs, lr_bvals, lr_index, b0_index = extract_single_shell(
        bvals, bvecs, bval, bval_range, dir_i
    )
    lr_index = np.array(lr_index.tolist())
    lr_bvals = np.array([0] + lr_bvals.tolist())
    lr_bvecs = np.concatenate([np.zeros((1, 3)), lr_bvecs])
    new_grad = np.concatenate([lr_bvecs, lr_bvals.reshape(-1, 1)], axis=-1)

    mif_b0 = np.mean(mif.data[..., b0_index], axis=-1, keepdims=True)
    f_data = np.concatenate([mif_b0, mif.data[..., lr_index]], axis=-1)

    mif.data = f_data
    mif.grad = new_grad

    print(' '.join([' '] * 300), end='\r')
    print('Converting to MIF (Downsampled) {:} [{:02d}/{:02d}]'.format(
        sub, i + 1, len(dwi_files)
    ), end='\r')
    out_name = os.path.join(
        path, '{:}_{:}dir.mif.gz'.format(
            image.split('.')[0], dir_i
        )
    )
    mif.save(out_name)